In [1]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
file_path='data/BreastCancer/breast-cancer-wisconsin.data.txt'
data=pd.read_csv(file_path, delimiter=',',header =None,skipinitialspace=True)

data.head()
#data.describe()
#data.info()

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [2]:
data=data.replace('?', np.NaN)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
0     699 non-null int64
1     699 non-null int64
2     699 non-null int64
3     699 non-null int64
4     699 non-null int64
5     699 non-null int64
6     683 non-null object
7     699 non-null int64
8     699 non-null int64
9     699 non-null int64
10    699 non-null int64
dtypes: int64(10), object(1)
memory usage: 60.1+ KB


In [3]:
data=data.dropna()
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
0     683 non-null int64
1     683 non-null int64
2     683 non-null int64
3     683 non-null int64
4     683 non-null int64
5     683 non-null int64
6     683 non-null object
7     683 non-null int64
8     683 non-null int64
9     683 non-null int64
10    683 non-null int64
dtypes: int64(10), object(1)
memory usage: 64.0+ KB


In [4]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X=LabelEncoder()
data.iloc[:,10] = labelencoder_X.fit_transform(data.iloc[:,10])

#data.head(20)

In [5]:
from sklearn.model_selection import train_test_split
x_train_raw, x_test_raw,y_train,y_test = train_test_split(data.iloc[:,1:10], data.iloc[:,10],test_size=0.3, random_state=41)

m=x_train_raw.shape[0]
n=x_test_raw.shape[0]

x_train = np.c_[np.ones((m, 1)), x_train_raw]
y_train=y_train.reshape(-1, 1)

x_test = np.c_[np.ones((n, 1)), x_test_raw]
y_test=y_test.reshape(-1, 1)

#print(x_train)
#print(y_train)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()


In [6]:
def random_batch(x_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(x_train), batch_size)
    x_batch = x_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return x_batch, y_batch

In [7]:
# Parameters
learning_rate = 0.01
display_step = 1
n_inputs=9

x = tf.placeholder(tf.float32, shape=(None, n_inputs + 1), name="x")
y = tf.placeholder(tf.int64, shape=(None), name="y")
theta = tf.Variable(tf.random_uniform([n_inputs + 1, 1], -1.0, 1.0, seed=42), name="theta")
logits = tf.matmul(x, theta, name="logits")
y_proba = tf.sigmoid(logits)

cost = tf.losses.log_loss(y, y_proba)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [8]:
n_epochs = 100
batch_size = 50
n_batches = int(np.ceil(len(x_train) / batch_size))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch, y_batch = random_batch(x_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={x: x_batch, y: y_batch})
        loss_val = cost.eval({x: x_test, y: y_test})
        if epoch % 5 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val)

    y_proba_val = y_proba.eval(feed_dict={x: x_test, y: y_test})

('Epoch:', 0, '\tLoss:', 1.5123576)
('Epoch:', 5, '\tLoss:', 1.0427825)
('Epoch:', 10, '\tLoss:', 0.79923826)
('Epoch:', 15, '\tLoss:', 0.6768658)
('Epoch:', 20, '\tLoss:', 0.60484755)
('Epoch:', 25, '\tLoss:', 0.5515249)
('Epoch:', 30, '\tLoss:', 0.51775426)
('Epoch:', 35, '\tLoss:', 0.49330413)
('Epoch:', 40, '\tLoss:', 0.47511488)
('Epoch:', 45, '\tLoss:', 0.45404065)
('Epoch:', 50, '\tLoss:', 0.44357812)
('Epoch:', 55, '\tLoss:', 0.4363947)
('Epoch:', 60, '\tLoss:', 0.4231846)
('Epoch:', 65, '\tLoss:', 0.41881287)
('Epoch:', 70, '\tLoss:', 0.4132993)
('Epoch:', 75, '\tLoss:', 0.40548807)
('Epoch:', 80, '\tLoss:', 0.393409)
('Epoch:', 85, '\tLoss:', 0.3886589)
('Epoch:', 90, '\tLoss:', 0.3819426)
('Epoch:', 95, '\tLoss:', 0.38153356)


In [9]:
#print(y_proba_val)
y_pred = (y_proba_val >= 0.5)
#print(y_pred)

from sklearn import metrics
score_test = metrics.accuracy_score(y_test, y_pred)
print(score_test)

0.8975609756097561
